In [2]:
import itertools
import pandas as pd
import pulp

In [3]:
def restructure_voorkeuren(df):
    df = df.melt(ignore_index=False, var_name='Type', value_name='Waarde').dropna()
    df[['Type', 'Nr']] = df['Type'].str.split('.', expand=True).fillna('0')
    df = df.set_index(['Type', 'Nr'], append=True)
    return df

In [4]:
leerlingen = pd.read_excel('voorkeuren.xlsx', index_col=0).drop(columns=['GRAAG_MET', 'GRAAG_MET.1'])
voorkeuren = leerlingen.pipe(restructure_voorkeuren)

In [6]:
M = 1_000_000 # A very big number, so that constraints are never larger than 1
prob = pulp.LpProblem('leerlingindeling', pulp.LpMaximize)
in_oranje = pulp.LpVariable.dict('InOranje', leerlingen.index, 0, 1, 'Binary')
in_blauw = pulp.LpVariable.dict('InBlauw', leerlingen.index, 0, 1, 'Binary')
in_geel = pulp.LpVariable.dict('InGeel', leerlingen.index, 0, 1, 'Binary')
in_groen = pulp.LpVariable.dict('InGroen', leerlingen.index, 0, 1, 'Binary')
dct = {'Blauw': in_blauw, 'Groen': in_groen, 'Oranje': in_oranje, 'Geel': in_geel}

# Every student must be in exactly one group
for i in leerlingen.index:
    prob += pulp.lpSum([groep[i] for groep in dct.values()]) == 1

# Every group can have a max number of students from an earlier group (no kliekjes)
for groep in dct.values():
    prob += pulp.lpSum(groep) <= 5
    # prob += pulp.lpSum(groep) >= 2

# Some students can not move int other groups (e.g. a brother/sister is already there)
for i, row in voorkeuren.query('Type == "Niet in"').iterrows():
    ll, type_, nr = i
    groep = dct[row['Waarde']]
    prob += groep[ll] == 0

# Now it's really starting: who prefers to be with whom - this we want to optimize
graag_met = voorkeuren.xs('Graag met', level='Type')
satisfied = pulp.LpVariable.dicts("Satisfied", graag_met.index.to_list(), cat="Binary")
satisfied_per_group = pulp.LpVariable.dicts("Satisfied_per_group", list(itertools.chain(*[[(t[0], t[1], c) for c in dct.keys()] for t in graag_met.index])), cat="Binary")
for i, row in graag_met.iterrows():
    ll, nr = i
    if row['Waarde'] not in dct.keys():
        andere_ll = row['Waarde']
        for groepsnaam, groep in dct.items():
            # Matching preferences are an XNOR problem, see https://yetanothermathprogrammingconsultant.blogspot.com/2022/06/xnor-as-linear-inequalities.html
            prob += satisfied_per_group[(ll, nr, groepsnaam)] >= 1 - groep[ll] - groep[andere_ll]  # Allebei niet in deze groep ==> satisfied = 1
            prob += satisfied_per_group[(ll, nr, groepsnaam)] <= 1 + groep[ll] - groep[andere_ll]  # ll niet in groep, ander wel ==> satisfied = 0
            prob += satisfied_per_group[(ll, nr, groepsnaam)] <= 1 - groep[ll] + groep[andere_ll]  # ll in groep, ander niet ==> satisfied = 0
            prob += satisfied_per_group[(ll, nr, groepsnaam)] >= groep[ll] + groep[andere_ll] - 1  # allebei in deze groep ==> satisfied = 1

            # AND definition: see https://yetanothermathprogrammingconsultant.blogspot.com/2022/06/xnor-as-linear-inequalities.html
            prob += satisfied[i] <= satisfied_per_group[(ll, nr, groepsnaam)]  # The total preference is only satisfied if it is at least correct for this group
        prob += satisfied[i] >= pulp.lpSum([satisfied_per_group[(ll, nr, groepsnaam)] for groepsnaam in dct.keys()]) - len(dct) + 1  # The preference is satisfied if it is correct for every group
    else:
        groep = dct[row['Waarde']]
        prob += (groep[ll] >= satisfied[i])

# We do not want to optimize the number of matches: at least 1 match for a student is more valuable than the third
satisfaction_per_ll = pulp.LpVariable.dict("LLSatisfaction", leerlingen.index, cat='Continuous')
n_true_per_ll = pulp.LpVariable.dicts("llassignedprefs", itertools.product(leerlingen.index, [str(i) for i in range(1, 6)]), cat='Binary')
for ll in leerlingen.index:
    ll_prefs = []
    for i in range(5):
        try:
            ll_prefs.append(satisfied[(ll, str(i))])
        except KeyError:
            break
    
    # The division works in combination with the fact that n_true_per_ll is binary
    prob += n_true_per_ll[(ll, '1')] <= pulp.lpSum(ll_prefs)
    prob += n_true_per_ll[(ll, '1')] >= (pulp.lpSum(ll_prefs) - 0.5) / M  # M ensures the constraint is never larger than 1
    prob += n_true_per_ll[(ll, '2')] <= pulp.lpSum(ll_prefs) / 2
    prob += n_true_per_ll[(ll, '2')] >= (pulp.lpSum(ll_prefs) - 1.5) / M
    prob += n_true_per_ll[(ll, '3')] <= pulp.lpSum(ll_prefs) / 3
    prob += n_true_per_ll[(ll, '3')] >= (pulp.lpSum(ll_prefs) - 2.5) / M
    prob += n_true_per_ll[(ll, '4')] <= pulp.lpSum(ll_prefs) / 4
    prob += n_true_per_ll[(ll, '4')] >= (pulp.lpSum(ll_prefs) - 3.5) / M
    prob += n_true_per_ll[(ll, '5')] <= pulp.lpSum(ll_prefs) / 5
    prob += n_true_per_ll[(ll, '5')] >= (pulp.lpSum(ll_prefs) - 4.5) / M

    satisfaction_per_ll[ll] = 1 * n_true_per_ll[(ll, '1')] + 0.5 * n_true_per_ll[(ll, '2')] + 0.25 * n_true_per_ll[(ll, "3")] + 0.125 * n_true_per_ll[(ll, "4")] + 0.0625 * n_true_per_ll[(ll, "5")]


prob += pulp.lpSum(satisfaction_per_ll)
# prob += pulp.lpSum(satisfied)

prob.solve()
print(pulp.LpStatus[prob.status])

Optimal


In [7]:
outcome = [v.name for v in prob.variables() if v.value() == 1 and v.name.startswith('In')]
outcome = pd.DataFrame([o.split('_') for o in outcome]).sort_values(0)
for groepsnaam in dct.keys():
    groepsleerlingen = outcome.loc[lambda df: df[0].str.contains(groepsnaam.title()), 1].tolist()
    print(f'Naar {groepsnaam}:', end='\n  - ')
    print('\n  - '.join(groepsleerlingen))

Naar Blauw:
  - Ca
  - Se
  - St
Naar Groen:
  - Ms
  - Mu
  - Ra
Naar Oranje:
  - El
  - Ro
  - Su
Naar Geel:
  - Ee
  - Fl
  - Li
  - Na
  - Ri


In [9]:
cs = pd.Series({v.name: v.value() for v in prob.variables() if v.name.startswith('Satisfied') and not 'per_group' in v.name}).to_frame()
ll_satisfaction = pd.Series({ll: satisfaction_per_ll[ll].value() for ll in leerlingen.index}, name='Satisfaction')
ix = cs.reset_index()['index'].str.extract("Satisfied_\('(.*)',(.*)\)").rename(columns={0: 'll', 1:'Nr'}).set_index(['ll', 'Nr']).index
cs.index = ix
per_ll = cs.groupby('ll')[0].agg(['count','sum', 'mean']).join(ll_satisfaction)
max_satisfaction_per_n_wishes = {1: 1, 2: 1.5, 3: 1.75, 4:1.875, 5: 1.9325}
max_attainable_satisfaction = (per_ll['count'].value_counts() # nr of ll with n_wishes
                               .to_frame()
                               .assign(max_satisfaction = max_satisfaction_per_n_wishes, # value per student with n_wishes
                                       prod = lambda df: df['count'] * df['max_satisfaction']) # max satisfaction per n_wishes
                                       ['prod'].sum() # sum to get class total
                                       )

display(pd.concat([cs.agg(['count', 'sum', 'mean']), pd.Series({'Satisfaction': per_ll['Satisfaction'].sum() / max_attainable_satisfaction})]).transpose())
display(per_ll)

,count,sum,mean,Satisfaction
0,32.0,22.0,0.6875,0.849758


,count,sum,mean,Satisfaction
ll,,,,
Ca,2,2.0,1.000000,1.5
Ee,3,2.0,0.666667,1.5
El,1,1.0,1.000000,1.0
Fl,2,1.0,0.500000,1.0
Li,2,1.0,0.500000,1.0
Ms,3,2.0,0.666667,1.5
Mu,2,2.0,1.000000,1.5
Na,3,2.0,0.666667,1.5
Ra,2,1.0,0.500000,1.0


Optimaliseer naar leerlingtevredenheid:
* Max 6 per groep: 23 wensen vervuld, 86.2% (niemand naar Oranje)
* Max 5 per groep: 22 wensen vervuld, 85.0% (Ro 3e wens niet)
* Max 4 per groep: 21 wensen vervuld, 82.6% 

Optimaliseer naar aantal wensen vervuld
* Max 6 per groep: 25 wensen vervuld, 83.8% (El, Su geen wensen)
* Max 5 per groep: 23 wensen vervuld, 80.3% (El, Fl geen wensen)
* Max 4 per groep: 21 wensen vervuld, 79.1% (Fl geen wensen)